In [ ]:
# display output from every line without explicit print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pathlib
import pandas as pd
import plotnine as pn

# Introduction to Python for Data Science, Homework 3

YOUR NAME HERE

# Question 1

This problem set will use data visualization to draw various insights from the Census ABS data. However, visualizing 51 states and 100 industries is difficult. Therefore, in this question, you will first pull in auxiliary information to help categorize states into regions and industries into sector groups. Questions 2 and 3 will then create the plots.

## 1.a

Load `census_abs.csv` into an object named `abs_df`. Drop duplicates. You should have 5,027 rows.

Next, load `state_pop.csv` into an object named `pop_df`. Merge population into `abs_df`. 

Verify that you still have 5,027 rows and the resulting `DataFrame` has no missingness. If you need to modify any part of `pop_df`, do it through code (i.e., do not edit the underlying CSV).

In [ ]:
abs_path = pathlib.Path('./census_abs.csv')
abs_df = pd.read_csv(abs_path)
abs_df = abs_df.drop_duplicates()
abs_df['state'] = abs_df['state'].replace('District of Columbia', 'Washington, D.C.')

state_path = pathlib.Path('./state_pop.csv')
pop_df = pd.read_csv(state_path)
pop_df.columns = ['state', 'state_population']
pop_df = pop_df[pop_df['state'].isin(abs_df['state'])]

abs_df = abs_df.merge(pop_df, on='state')


## 1.b

Programmatically convert this information on Census Bureau-designated regions and divisions, which is provided below in its original structure as found on https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States, into a `DataFrame`. Merge it into `abs_df`.

Verify that you still have 5,027 rows and the resulting `DataFrame` has no missingness. If you need to modify any part of `regions_dict`, do it through code (i.e., do not edit the code block below).

**Hint:** One approach is to loop through regions and, for each region, loop through divisions. For each division, you can then construct a division-specific `DataFrame`. Finally, concatenate the results with `pd.concat()`.

In [ ]:
# do not edit this code block
regions_dict = {
    # region 1
    'Northeast' : {
        # division 1
        'New England' : ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'],
        # division 2
        'Mid-Atlantic' : ['New Jersey', 'New York', 'Pennsylvania']
    },
    # region 2
    'Midwest' : {
        # division 3
        'East North Central' : ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin'],
        # division 4
        'West North Central' : ['Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota']
    },
    # region 3
    'South' : {
        # division 5
        'South Atlantic' : ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'Washington, D.C.', 'West Virginia'],
        # division 6
        'East South Central' : ['Alabama', 'Kentucky', 'Mississippi', 'Tennessee'],
        # division 7
        'West South Central' : ['Arkansas', 'Louisiana', 'Oklahoma', 'Texas']
    },
    # region 4
    'West' : {
        # division 8
        'Mountain' : ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming'],
        # division 9
        'Pacific' : ['Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']        
    }
}

In [ ]:
regions_df = pd.DataFrame()
for region, divisions in regions_dict.items():  #used outside reference
    for division, states in divisions.items():  #used outside reference
        division_df = pd.DataFrame({'region': region, 'division': division, 'state': states})  #used outside reference
        regions_df = pd.concat([regions_df, division_df])  #used outside reference
regions_df = regions_df.reset_index(drop=True)   #used outside reference

abs_df = abs_df.merge(regions_df, on='state')  #used outside reference

## 1.c

Load `census_abs_sector_industry.csv`, which provides a sector code for each industry, into an object called `sector_df`. Define your own categorization scheme to group these into 8–12 meaningful groups. Then, come up with a concise but informative name for each group.

Merge your group names into `abs_df`. You are free to drop rows that contain totals or catch-all "other" industries.

**Hint:** There are many ways to reduce/combine the 21 sector codes into 8–12 sector groups. Here are three possible strategies to get you started. You will probably only need one of these.
- `.cat`
- `.rename()`
- `merge()`

In [ ]:
sector_path = pathlib.Path('./census_abs_sector_industry.csv')
sector_df = pd.read_csv(sector_path)
sector_df['sector_code'] = sector_df['sector_code'].replace([10, 11, 12, 13, 14, 15, 16, 17, 18, 19], '10-series') #used outside reference
sector_df['sector_code'] = sector_df['sector_code'].replace([20, 21, 22, 23, 24, 25, 26, 27, 28, 29], '20-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([30, 31, 32, 33, 34, 35, 36, 37, 38, 39], '30-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([40, 41, 42, 43, 44, 45, 46, 47, 48, 49], '40-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([50, 51, 52, 53, 54, 55, 56, 57, 58, 59], '50-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([60, 61, 62, 63, 64, 65, 66, 67, 68, 69], '60-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([70, 71, 72, 73, 74, 75, 76, 77, 78, 79], '70-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([80, 81, 82, 83, 84, 85, 86, 87, 88, 89], '80-series')
sector_df['sector_code'] = sector_df['sector_code'].replace([90, 91, 92, 93, 94, 95, 96, 97, 98, 99], '90-series')
display(sector_df)


abs_df = abs_df[abs_df['industry'] != 'Total for all sectors']  #used outside reference
abs_df = abs_df.merge(sector_df, on='industry')


display(abs_df)



# Question 2

In this question, you will make scatterplots.

## 2.a

Make a scatterplot in which each point is a state-industry, the horizontal axis is the number of employees (in thousands), and the vertical axis is the total payroll (in billions of dollars). Add well-formatted and informative axis labels. Pick a nice theme from https://plotnine.readthedocs.io/en/stable/generated/plotnine.themes.theme.html and apply it.

In [ ]:

pn.options.figure_size = (10, 10)

(
    pn.ggplot(abs_df,
              pn.aes(x = 'n_employees',
                     y = 'payroll_1000s',
                     color = 'sector_code',
                    )
             )
    + pn.geom_point()
    + pn.xlab('Number of Employees (thousands)')
    + pn.ylab('Payrool ($billions)')
    + pn.scale_y_continuous(labels = lambda x: x/1000000)  #used outside reference
    + pn.scale_x_continuous(labels = lambda x: x/1000)  #used outside reference
)


## 2.b

First, compute the average wage by dividing total payroll (across all states and industries) by the total number of employees.

Second, take your previous plot and facet it by sector group.

Third, using `geom_abline()`, add a dashed line to each facet to indicate where we would expect points to lie if every worker received the same annual wage. You should use `intercept = 0` and provide a `slope` argument based on your calculations from above.

Finally, using `ggtitle()`, add an informative title that makes a claim about how individuals in one particular sector are paid more than, less than, or about the same as the average wage.

In [ ]:
total_payroll = abs_df['payroll_1000s'].sum()
total_employees = abs_df['n_employees'].sum()
avg_wage = total_payroll / total_employees
avg_wage

(
    pn.ggplot(abs_df,
              pn.aes(x = 'n_employees',
                     y = 'payroll_1000s',
                     color = 'sector_code',
                    )
             )
    + pn.geom_point()
    + pn.xlab('Number of Employees (thousands)')
    + pn.ylab('Payrool ($billions)')
    + pn.scale_y_continuous(labels = lambda x: x/1000000)
    + pn.scale_x_continuous(labels = lambda x: x/1000)
    + pn.facet_wrap('sector_code')
    + pn.geom_abline(intercept = 0, slope = avg_wage, linetype = 'dashed')
    + pn.ggtitle('Each sector\'s wages in comparison to the average wage (dashed line)')
)

## 2.c

First, create a new `wage` column in `abs_df`, containing the average wage within each state-industry.

Second, create a scatterplot in which the horizontal axis is sector, the vertical axis is average wage, and color is region. 
- Use `position = pn.position_dodge(width = .5)` to offset points from different regions so they do not overlap. 
- Use `scale_color_manual()` or `scale_color_brewer()` to add a custom color scheme. If you use ColorBrewer, some palette names can be found at www.colorbrewer.org. 
- Use `theme(axis_text_x = ...)` to tweak your labels so they do not overlap. The `axis_text_x` argument must be generated using `element_text()`. Some useful arguments for `element_text()` are `angle`, `hjust`, and `vjust`. You should consult the documentation and play around with settings to find a solution that works.

Finally, as always, use well-formatted and informative labels. Add an informative title that makes a claim about whether region or sector group is a better predictor of wages.

In [ ]:
abs_df['wage'] = abs_df['payroll_1000s'] / abs_df['n_employees']

(
    pn.ggplot(abs_df,
                pn.aes(x = 'sector_code',
                       y = 'wage',
                       color = 'region',
                       )                                                               
                )
    + pn.geom_point(position = pn.position_dodge(width = 0.5))
    + pn.scale_color_brewer()
    + pn.xlab('Sector')
    + pn.theme(axis_text_x = pn.element_text(angle = 90, hjust = 1))  #used outside reference
    + pn.ggtitle('Wages by sector and region')
)
    


# Question 3

In this question, you will make other kinds of plots.

## 3.a

First, aggregate your data to the region level. Compute the average wage of all workers in the region—that is, total payroll (summing over all state-industries in the region), divided by total number of workers (again summing over all state-industries in the region).

Then, create a bar plot, using `geom_col()`, in which the vertical height of each bar is the average wage. The horizontal axis should denote region. You should have a total of four bars.

Using `scale_fill_brewer()` or `scale_fill_manual()`, fill each bar with a different color. Add a black border to each bar. Using `geom_hline()`, add a solid black line at zero. As always, use well-formatted and informative labels.

In [ ]:

region_df = abs_df.groupby(['region']).agg({'payroll_1000s': 'sum', 'n_employees': 'sum'}).reset_index()  #used outside reference
region_df['avg_wage'] = region_df['payroll_1000s'] / region_df['n_employees']  #used outside reference

(
    pn.ggplot(region_df,
                pn.aes(x = 'region',
                       y = 'avg_wage',   
                      )
                )
    + pn.geom_col()
    + pn.xlab('Region')
    + pn.ylab('Average Wage')
    + pn.ggtitle('Average Wage by Region')
    + pn.scale_fill_manual(values = ['#FF0000', '#FF7F00', '#FFFF00', '#00FF00', '#0000FF', '#4B0082', '#9400D3'])
    + pn.geom_hline(yintercept = 0, linetype = 'solid', color = 'black')  #used outside reference 
    + pn.theme(axis_text_x = pn.element_text(angle = 90, hjust = 1))
)


## 3.b

First, aggregate your data to regions and sector groups. Within each region and sector group, compute the average wage.

Second, group by region. Programmatically identify whether a sector group is the highest-paying in its region. There should be four `True` values in this output. Store this information as an additional column in your aggregated data.

Third, create a plot using `geom_text()` in which the horizontal axis is region, the vertical axis is average wage, and the label is the name of the sector group. Fiddle with plot settings until your text is legible and does not overlap too much (some vertical overlap is inevitable, because there are sector groups in each region that pay a similar amount).

Finally, using `scale_color_manual()`, indicate the highest-paying industry in each region by coloring it in red. Leave all other industries in the region black. As always, use well-formatted and informative labels.

In [ ]:

region_sector_df = abs_df.groupby(['region', 'sector_code']).agg({'payroll_1000s': 'sum', 'n_employees': 'sum'}).reset_index()  #used outside reference
region_sector_df['avg_wage'] = region_sector_df['payroll_1000s'] / region_sector_df['n_employees']  #used outside reference

region_sector_df['highest_wage'] = region_sector_df.groupby('region')['avg_wage'].transform(max) == region_sector_df['avg_wage']  #used outside reference

(
    pn.ggplot(region_sector_df,
                pn.aes(x = 'region',
                       y = 'avg_wage',
                       label = 'sector_code',
                            )
                )
    + pn.geom_text(position = pn.position_dodge(width = 0.5))
    + pn.xlab('Region')
    + pn.ylab('Average Wage')
    + pn.ggtitle('Average Wage by Region')
    + pn.geom_hline(yintercept = 0, linetype = 'solid', color = 'black')
    + pn.theme(axis_text_x = pn.element_text(angle = 90, hjust = 1))
    + pn.scale_color_manual(values = ['#000000', '#FF0000'])

)

